In [1]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")


# --- Wikipedia Search Tool ---
class WikipediaSearchArgs(BaseModel):
    query: str = Field(description="Search query for Wikipedia.")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearch"
    description = "Use this tool to search and get a summary from Wikipedia."
    args_schema: Type[WikipediaSearchArgs] = WikipediaSearchArgs

    def _run(self, query: str):
        try:
            wiki = WikipediaAPIWrapper()
            return wiki.run(query)
        except Exception as e:
            return f"Wikipedia error: {e}"


# --- DuckDuckGo Search Tool ---
class DuckDuckGoSearchArgs(BaseModel):
    query: str = Field(description="Search query for DuckDuckGo.")


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearch"
    description = "Use this tool to search and get a summary from DuckDuckGo."
    args_schema: Type[DuckDuckGoSearchArgs] = DuckDuckGoSearchArgs

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


# --- Web Scraping Tool ---
class WebScrapingArgs(BaseModel):
    url: str = Field(description="The full URL of the web page to scrape.")


class WebScrapingTool(BaseTool):
    name = "WebScrapingTool"
    description = "Extracts visible text from a given web page"
    args_schema: Type[WebScrapingArgs] = WebScrapingArgs

    def _run(self, url: str):
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
            combined_text = "\n".join([doc.page_content for doc in docs])
            return combined_text[:3000]  # LLM 입력 제한 고려
        except Exception as e:
            return f"Web scraping failed: {e}"


# --- Save to Text File Tool ---
class SaveToTextFileArgs(BaseModel):
    content: str = Field(description="The research content to save.")


class SaveToTextFileTool(BaseTool):
    name = "SaveToTextFile"
    description = "Save the research result to a .txt file"
    args_schema: Type[SaveToTextFileArgs] = SaveToTextFileArgs

    def _run(self, content: str):
        with open("research_output.txt", "w", encoding="utf-8") as f:
            f.write(content)
        return "Research saved to research_output.txt"


# 5. 에이전트 초기화
tools = [
    WikipediaSearchTool(),
    DuckDuckGoSearchTool(),
    WebScrapingTool(),
    SaveToTextFileTool(),
]

agent = initialize_agent(
    llm=llm, tools=tools, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (pr

{'input': 'Research about the XZ backdoor',
 'output': 'I found information about the XZ backdoor. In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score. While xz is commonly present in most Linux distributions, at the time of discovery, the backdoored version had not yet been widely deployed to production systems but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.\n\nI have saved this information to a file named "research_output.txt".'}